In [1]:
# import necessary modules
import os
import json
import pandas as pd
from rdkit import Chem
import rdkit.Chem.rdFreeSASA as FreeSASA
from pymol import cmd
import multiprocessing as mp

In [2]:
# Defined constants:
PATH_TO_BASE_FOLDER = '/home/fol007/PycharmProjects/ChEMBL_plus_BindingMOAD/BindingMOAD_AstexDiverseSet_Simplified' #'/BindingMOAD_AstexDiverseSet_Simplified'
PATH_TO_PDB_FOLDER = f'{PATH_TO_BASE_FOLDER}/pdb_files'
PATH_TO_REFERENCE_LIGANDS_FOLDER = f'{PATH_TO_BASE_FOLDER}/reference_ligands'

In [3]:
# Helper functions:

def create_reference_dictionary(path_to_reference_ligands_folder=PATH_TO_REFERENCE_LIGANDS_FOLDER):
    '''organize dictionary based on the reference ligands'''
    reference_dictionary = {}
    uniprot_ids = os.listdir(path_to_reference_ligands_folder)
    for uniprot_id in uniprot_ids:
        reference_dictionary[uniprot_id] = []
        for references in os.listdir(path_to_reference_ligands_folder + '/' + uniprot_id):
            reference_dictionary[uniprot_id] += [references]
    return reference_dictionary

# Function to classify atoms as polar or apolar
def classifyAtoms(mol, polar_atoms=[7,8,15,16]):
    # the polar atoms are [N,O,S,P]
    # Get Van der Waals radii (angstrom)
    ptable = Chem.GetPeriodicTable()
    radii = [ptable.GetRvdw(atom.GetAtomicNum()) for atom in mol.GetAtoms()]

    for atom in mol.GetAtoms():
        atom.SetProp("SASAClassName", "Apolar") # mark everything as apolar to start
        if atom.GetAtomicNum() in polar_atoms: #identify polar atoms and change their marking
            atom.SetProp("SASAClassName", "Polar") # mark as polar
        elif atom.GetAtomicNum() == 1:
            try:
                if atom.GetBonds()[0].GetOtherAtom(atom).GetAtomicNum() in polar_atoms:
                    atom.SetProp("SASAClassName", "Polar") # mark as polar
            except:
                continue

    return radii

def calculate_sasa(path_to_mol, path_to_pdb):

    # Get mol of molecule:
    mol = Chem.SDMolSupplier(path_to_mol)[0]
    # Get mol atoms
    mol_atoms = mol.GetAtoms()
    # Get conformer of mol
    mol_conf = mol.GetConformer()

    # First I am going to calculate the SASA of the unbond molecule
    radii = classifyAtoms(mol)
    FreeSASA.CalcSASA(mol, radii)

    sasa_unbond_polar = sum(float(a.GetProp("SASA")) for a in mol_atoms if a.GetProp("SASAClassName")=='Polar')
    sasa_unbond_apolar = sum(float(a.GetProp("SASA")) for a in mol_atoms if a.GetProp("SASAClassName")=='Apolar')

    # Use pymol to obtain the bond ligand and the surrounding residues (minus the waters)
    cmd.load(path_to_mol, object='mol')
    cmd.load(path_to_pdb, object='pdb')

    cmd.remove(selection='resn hoh')

    cmd.select(name='bonded_mol', selection='br. pdb within 6.5 of mol')

    bonded_mol_path = path_to_mol.split('/')[-1]+'.pdb'
    cmd.save(filename=bonded_mol_path, selection='bonded_mol')

    cmd.reinitialize()

    bonded_mol = Chem.MolFromPDBFile(bonded_mol_path, sanitize=False)
    os.remove(bonded_mol_path)

    # Get mol atoms
    bonded_mol_atoms = bonded_mol.GetAtoms()
    # Get conformer of mol
    bonded_mol_conf = bonded_mol.GetConformer()

    # Calculate SASA of the bonded mol
    radii=classifyAtoms(bonded_mol)
    FreeSASA.CalcSASA(bonded_mol, radii)

    for i in range(len(bonded_mol_atoms)):

        coords_bonded = (bonded_mol_conf.GetAtomPosition(i).x,
                         bonded_mol_conf.GetAtomPosition(i).y,
                         bonded_mol_conf.GetAtomPosition(i).z)

        for j in range(len(mol_atoms)):

            coords_mol = (mol_conf.GetAtomPosition(j).x, mol_conf.GetAtomPosition(j).y, mol_conf.GetAtomPosition(j).z)

            if coords_bonded == coords_mol:

                mol_atoms[j].SetProp("SASA_bonded", bonded_mol_atoms[i].GetProp("SASA"))

    sasa_bond_polar = sum(float(a.GetProp("SASA_bonded")) for a in mol_atoms if a.GetProp("SASAClassName")=='Polar')
    sasa_bond_apolar = sum(float(a.GetProp("SASA_bonded")) for a in mol_atoms if a.GetProp("SASAClassName")=='Apolar')

    atom_properties = []
    for i in range(len(mol_atoms)):
        atom_properties += [{'SASAClassName' : mol_atoms[i].GetProp('SASAClassName'),
                             'SASA' : mol_atoms[i].GetProp('SASA'),
                             'SASA_bonded' : mol_atoms[i].GetProp('SASA_bonded')}]

    with open('../data/sasa_per_atoms/'+path_to_mol.split('/')[-1].split('.')[0]+'.json', 'w') as outfile:
        json.dump(atom_properties, outfile, indent=4)

    return sasa_bond_apolar, sasa_bond_polar, sasa_unbond_apolar, sasa_unbond_polar

def run_in_parallel(index, path_to_mol, path_to_pdb):
    try:
        sasa_bond_apolar, sasa_bond_polar, sasa_unbond_apolar, sasa_unbond_polar = calculate_sasa(path_to_mol, path_to_pdb)
        error = -1
    except Exception as e:
        sasa_bond_apolar, sasa_bond_polar, sasa_unbond_apolar, sasa_unbond_polar = None, None, None, None
        error = e
    return (index, sasa_bond_apolar, sasa_bond_polar, sasa_unbond_apolar, sasa_unbond_polar, error)

In [4]:
reference_dictionary = create_reference_dictionary()

dictionary_df = {'uniprot_id':[], 'reference':[]}

for uniprot_id in reference_dictionary:
    for ref in reference_dictionary[uniprot_id]:
        dictionary_df['uniprot_id'] += [uniprot_id]
        dictionary_df['reference'] += [ref.split('.')[0]]

dictionary_df = pd.DataFrame(dictionary_df)

dictionary_df['sasa_bond_polar'] = None
dictionary_df['sasa_bond_apolar'] = None
dictionary_df['sasa_unbond_polar'] = None
dictionary_df['sasa_unbond_apolar'] = None
dictionary_df['error'] = None

dictionary_df.head(3)

,uniprot_id,reference,sasa_bond_polar,sasa_bond_apolar,sasa_unbond_polar,sasa_unbond_apolar,error
0,P16083,3OVM_MZC,None,None,None,None,None
1,P16083,3G5M_XM5,None,None,None,None,None
2,P16083,3OWH_52X,None,None,None,None,None


In [5]:
work = []
#i = 0
for index, uniprot_id, ref in dictionary_df[['uniprot_id', 'reference']].itertuples():

    path_to_mol = PATH_TO_REFERENCE_LIGANDS_FOLDER + '/' + uniprot_id + '/' + ref + '.sdf'
    path_to_pdb = PATH_TO_PDB_FOLDER + '/' + uniprot_id + '/' + ref.split('_')[0] + '.pdb'

    work += [(index, path_to_mol, path_to_pdb)]

    #i += 1
    #if i ==10:
    #    break

print(len(work))

# Get process results from the output queue
pool = mp.Pool(processes=5)
results = pool.starmap(run_in_parallel, work)
print(len(results))

for index, sasa_bond_apolar, sasa_bond_polar, sasa_unbond_apolar, sasa_unbond_polar, error in results:
    dictionary_df.at[index, 'sasa_bond_apolar'] = sasa_bond_apolar
    dictionary_df.at[index, 'sasa_bond_polar'] = sasa_bond_polar
    dictionary_df.at[index, 'sasa_unbond_apolar'] = sasa_unbond_apolar
    dictionary_df.at[index, 'sasa_unbond_polar'] = sasa_unbond_polar
    dictionary_df.at[index, 'error'] = error

dictionary_df.to_csv('../data/total_sasa.csv', index=False)

2046
 PyMOL not running, entering library mode (experimental) PyMOL not running, entering library mode (experimental)
 PyMOL not running, entering library mode (experimental) PyMOL not running, entering library mode (experimental)

 PyMOL not running, entering library mode (experimental)

2046


In [6]:
dictionary_df.loc[dictionary_df['error']!=-1]



,uniprot_id,reference,sasa_bond_polar,sasa_bond_apolar,sasa_unbond_polar,sasa_unbond_apolar,error
76,P00742,1NFU_RRP,None,None,None,None,'SASA_bonded'
246,P28720,1K4H_APQ,None,None,None,None,'SASA_bonded'
1796,P03372,6IAR_H8W,None,None,None,None,'NoneType' object has no attribute 'GetAtoms'
